# Libraries Import

In [1]:
import pandas as pd
import numpy as np
import os.path
import recsys as rs
import notipy
import math
import sys
import TopSimilarRecommender as TSR
import random
from scipy import sparse as sps

# Datasets

In [2]:
train = pd.read_csv('Data/train_final.csv','\t')
tr_info = pd.read_csv('Data/tracks_final.csv','\t')
pl_info = pd.read_csv('Data/playlists_final.csv','\t')
tgt_pl = pd.read_csv('Data/target_playlists.csv','\t')
tgt_tr = pd.read_csv('Data/target_tracks.csv','\t')

# Data Visualization

In [3]:
train.head()

,playlist_id,track_id
0,3271849,2801526
1,5616275,727878
2,11267488,2805283
3,10103900,1515105
4,3836898,2945623


In [4]:
tr_info.head()

,track_id,artist_id,duration,playcount,album,tags
0,2972914,144,224000,49.0,[7],"[54087, 1757, 1718, 116712, 189631]"
1,2750239,246,157000,1.0,[8],"[189631, 3424, 177424, 46208, 205245]"
2,1550729,144,217000,554.0,[9],"[54087, 109806, 46869, 183258, 54337]"
3,2169950,144,207000,200.0,[9],"[54087, 70618, 207003, 109806, 116712]"
4,1903709,144,198000,5.0,[None],"[54087, 81223, 116712, 215342, 71028]"


In [5]:
pl_info.head()

,created_at,playlist_id,title,numtracks,duration,owner
0,1216545588,644838,[12727],27,6522,41504
1,1249326867,7577564,[],9,2650,41504
2,1257766688,3120683,[183],16,3645,44542
3,1248079275,4278112,"[12389, 18698, 18925, 11695, 7117]",15,4151,44542
4,1175201268,8656823,"[12809, 2095, 13257, 12671, 20426, 14448, 18698]",84,18414,44542


In [6]:
tgt_pl.head()

,playlist_id
0,10024884
1,10624787
2,4891851
3,4267369
4,65078


In [7]:
tgt_tr.head()

,track_id
0,1316175
1,3885714
2,3091270
3,226759
4,230596


In [8]:
train, test, tgt_tracks, tgt_playlists = rs.split_train_test(train, 10, 20, 5, 2517)

In [9]:
rec = TSR.TopSimilarRecommender()
fit_dict = {'tracks_info' : tr_info, 
            'attributes' : ['artist_id', 'album', 'tags', 'playcount'],
            'attributes_to_prune' : ['tags'],
            'tgt_tracks' : tgt_tracks,
            'n_min_attr' : 5,
            'idf' : True,
            'measure' : 'dot',
            'shrinkage' : 0,
            'n_el_sim' : 20}

recommend_dict = {'tgt_playlists' : tgt_playlists,
                  'train_playlists_tracks_pairs' : train, 
                  'normalize' : False}

In [10]:
rec.fit_without_matrix_builder(**fit_dict)
print('Model fitted!')

Fixed dataset
Calculated Indices
Eliminated low frequency attributes!
Model fitted!


In [11]:
_, _, rec.IX_tgt_playlists, _ = rs.create_sparse_indexes(playlists=tgt_playlists)
URM = rs.create_tgt_URM(rec.IX_tgt_playlists, rec.IX_items, train)
URM = URM.tocsr()
print('URM built')

100%|██████████| 4420/4420 [00:07<00:00, 587.57it/s]


URM built


In [12]:
URM.tocsc()

<4420x100000 sparse matrix of type '<class 'numpy.int32'>'
	with 165758 stored elements in Compressed Sparse Column format>

In [13]:
data = np.array([],dtype='float32')
rows = np.array([],dtype='int32')
columns = np.array([],dtype='int32')
nitems =URM.shape[1]
n_el = 60
vec_pow = np.vectorize(math.pow)
vec_sqrt= np.vectorize(math.sqrt)
H = 20

(1, 100000)

# Build Item Similarity Matrix with Cosine 

In [27]:
if (rec.IX_tgt_items is not None and rec.IX_items is not None):
    rec_URM = URM[:,rec.IX_items.loc[rec.IX_tgt_items.index.values].values.flatten()]
    rec_URM = rec_URM.tocsc()
    h = rec.IX_tgt_items.index.values.size
else:
    rec_URM = URM
    h = nitems
sum_i_tot = np.array(vec_pow(URM.sum(axis=0),2))
sum_i = np.array(vec_pow(rec_URM.sum(axis=0),2)) #array containing the sum of the ratings for each item to the power of 2..
sum_i.shape

(1, 17161)

In [28]:
rec_URM.shape

(4420, 17161)

In [31]:
for i in range (nitems):
    den = vec_sqrt(sum_i*sum_i_tot[0,i]).flatten()
    res= (URM[:,i].T.dot(rec_URM)).toarray().flatten()
    res = res/(den + H + 1e-6)
    sort = np.argsort(res)[-n_el:].astype(np.int32)
    data = np.append(data, res[sort])
    rows = np.append(rows, np.array([i]*n_el,dtype='int32'))
    columns = np.append(columns, sort)
    if i%1000 == 0:
        print(i)
item_similarity = sps.coo_matrix((data,(rows,columns)), shape=(nitems, h))

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000


In [32]:
sps.save_npz('BuiltStructures/ItemSimilarityMatrix.npz',item_similarity)
item_similarity

<100000x17161 sparse matrix of type '<class 'numpy.float64'>'
	with 13091880 stored elements in COOrdinate format>

# Build Item Similarity Matrix with Adjusted Cosine

In [67]:
#TODO

# Recommendation

In [30]:
#import already created item_sim matrix
item_similarity =sps.load_npz('BuiltStructures/ItemSimilarityMatrix.npz').tocsc()

In [22]:
print(rec.IX_items.index)
print(rec.IX_items.index[0])

Int64Index([2972914, 2750239, 1550729, 2169950, 1903709, 2256817, 2561768,
             474864, 1378455, 1523190,
            ...
            1559115,  348847, 2839226, 2996304, 2278635, 3022399, 1320641,
            2584455, 2299706, 2739985],
           dtype='int64', length=100000)
2972914


In [55]:
recommendetions = np.array([])
den = np.array(item_similarity.sum(axis=0)).flatten()
print(den)
for i in rec.IX_tgt_playlists.values: 
    estimated_ratings = URM[i,:].dot(item_similarity).toarray().flatten()
    estimated_ratings = estimated_ratings/(den+ 1e-6)
    tgt_in_playlist = np.intersect1d(train[train['playlist_id'] == rec.IX_tgt_playlists.index.values[i]]['track_id'].values, rec.IX_tgt_items.index.values, assume_unique=True)
    estimated_ratings[rec.IX_tgt_items.loc[tgt_in_playlist].values] = 0
    #if(np.count_nonzero(estimated_ratings) < 5): sys.exit('Not enough similarity')
    #estimated_ratings= np.take(estimated_ratings, indexes) #errore probabile, indexes non è calcolato
    top5_ind = np.flip(np.argsort(estimated_ratings)[-5:], axis=0)
    top5 = rec.IX_tgt_items.index.values[top5_ind]
    recommendetions = np.append(recommendetions, rs.sub_format(top5))
    #add break_equality
    if i%1000 == 0:
        print(i)
rec_df = pd.DataFrame({'playlist_id' : rec.IX_tgt_playlists.index.values, 'track_ids' : recommendetions})

[ 21.23432598  18.2304658   21.25384219 ...,   8.0049799    4.97414981   0.        ]
0
1000
2000
3000
4000


In [ ]:
test.shape

# Evaluation

In [51]:
def evaluate(results, test, eval_metric='MAP'):
    if eval_metric == 'MAP':
        APs = results.apply(calculate_AP, axis=1, args=(test,))
        res = (APs.sum())/results.shape[0]
    return res

def calculate_AP(row, test):
    p_id = row['playlist_id']
    recs = np.fromstring(row['track_ids'], dtype=float, sep=' ')

    AP = 0
    rel_sum = 0
    n_rel_items = min(test[test['playlist_id'] == p_id].shape[0],5)
    for i in range(recs.size):
        rel = 1 if ((test['playlist_id'] == int(p_id)) & (test['track_id'] == recs[i])).any() else 0
        rel_sum += rel
        P = rel_sum/(i+1)
        AP += (P * rel)/n_rel_items

    return AP

In [56]:
map_eval = evaluate(rec_df, test, 'MAP')
print('Evaluation completed!')

Evaluation completed!


In [ ]:
run_data = {'recommender_type' : rec.__class__.__name__,
            'fit_parameters' : {'idf' : fit_dict['idf'],
                                'measure' : fit_dict['measure'],
                                'shrinkage' : fit_dict['shrinkage'],
                                'n_el_sim' : fit_dict['n_el_sim']},
            'evaluation_result' : map_eval}

In [64]:
with open('runs_data.json', 'a') as fp:
    json.dump(run_data, fp, indent=2)
    fp.write('\n')
print('Run data saved!')

NameError: name 'json' is not defined

In [57]:
map_eval

0.013833333333333378

0.023716440422322783